In [1]:
!pip install numpy pandas matplotlib seaborn
!pip install nltk Sastrawi wordcloud
!pip install scikit-learn imbalanced-learn
!pip install torch transformers
!pip install tqdm
!pip install ipython

# Download NLTK stopwords
import nltk
nltk.download('stopwords')

# Verify GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


PyTorch version: 2.6.0+cu124
GPU available: False
GPU name: None


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

2025-07-25 03:59:03.585753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753415943.875036      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753415943.958596      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load the dataset
df = pd.read_csv('/kaggle/input/test344/processed_sentimen_prabowonomics.csv')
df['text'] = df['text'].astype(str)

# Display basic info
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 samples:")
display(df.head())

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Check label distribution
print("\nLabel distribution:")
print(df['label'].value_counts())

Dataset shape: (3424, 4)

First 5 samples:


,text,label,label_num,cleaned_text
0,tunjangan dpr dan pemerintah yg harus di sunat...,neutral,1.0,tunjang dpr perintah sunat sedia mobil dinas m...
1,semoga korupsi di berantas total agar rakyat t...,positive,2.0,moga korupsi berantas total rakyat ragu
2,koropsi di desa tolong secepatnya di periksa b...,neutral,1.0,koropsi desa tolong cepat periksa bpk presiden...
3,sasaran efisiensi terbesar dengan lenyapkan co...,negative,0.0,sasar efisiensi besar lenyap coruptor sakit ut...
4,jogetin aja,negative,0.0,jogetin



Missing values per column:
text             0
label            6
label_num        6
cleaned_text    25
dtype: int64

Label distribution:
label
negative    2157
positive     764
neutral      497
Name: count, dtype: int64


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
# Function to get top words
def get_top_words(text_series, n=10):
    # Combine all text
    all_text = ' '.join(text_series).lower()
    
    # Tokenize and remove stopwords
    words = word_tokenize(all_text)
    indonesian_stopwords = set(stopwords.words('indonesian'))
    filtered_words = [word for word in words if word.isalpha() and word not in indonesian_stopwords]
    
    # Count word frequencies
    word_counts = Counter(filtered_words)
    return word_counts.most_common(n)

# Get top words for each sentiment
top_positive = get_top_words(df[df['label'] == 'positive']['cleaned_text'])
top_negative = get_top_words(df[df['label'] == 'negative']['cleaned_text'])
top_neutral = get_top_words(df[df['label'] == 'neutral']['cleaned_text'])

# Function to plot top words
def plot_top_words(word_counts, title):
    words, counts = zip(*word_counts)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(words), palette='viridis')
    plt.title(title)
    plt.xlabel('Frequency')
    plt.ylabel('Words')
    plt.show()

TypeError: sequence item 58: expected str instance, float found

In [ ]:
plot_top_words(top_positive, 'Top 10 Words in Positive Sentiment')
plot_top_words(top_negative, 'Top 10 Words in Negative Sentiment')
plot_top_words(top_neutral, 'Top 10 Words in Neutral Sentiment')

# Create a table of top words
top_words_table = pd.DataFrame({
    'Positive': [word[0] for word in top_positive],
    'Negative': [word[0] for word in top_negative],
    'Neutral': [word[0] for word in top_neutral]
})

print("\nTop Words by Sentiment:")
print(top_words_table)

In [ ]:
# Plot label distribution
plt.figure(figsize=(8, 5))
sns.countplot(x='label', data=df)
plt.title('Distribution of Sentiment Labels')
plt.xlabel('Sentiment Label')
plt.ylabel('Count')
plt.show()

# Generate word clouds for each sentiment
def generate_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=500,
                         background_color='white',
                         stopwords=set(stopwords.words('indonesian')),
                         min_font_size=10).generate(text)
    plt.figure(figsize=(8, 5), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(title)
    plt.tight_layout(pad=0)
    plt.show()

# Positive sentiment word cloud
positive_text = " ".join(df[df['label'] == 'positive']['cleaned_text'])
generate_wordcloud(positive_text, "Positive Sentiment Word Cloud")

# Negative sentiment word cloud
negative_text = " ".join(df[df['label'] == 'negative']['cleaned_text'])
generate_wordcloud(negative_text, "Negative Sentiment Word Cloud")

# Neutral sentiment word cloud
neutral_text = " ".join(df[df['label'] == 'neutral']['cleaned_text'])
generate_wordcloud(neutral_text, "Neutral Sentiment Word Cloud")

In [ ]:
# Initialize Indonesian stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load Indonesian stopwords
nltk.download('stopwords')
indonesian_stopwords = set(stopwords.words('indonesian'))
custom_stopwords = {"yg", "dg", "tdk", "ny", "aja", "sih", "nya", "kok", "gue", "lo", "gw"}
combined_stopwords = indonesian_stopwords.union(custom_stopwords)

# Custom preprocessing function for Indonesian text
def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove user mentions and hashtagsa
    text = re.sub(r'@\w+|\#\w+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in combined_stopwords]

    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]

    # Join tokens back to string
    text = ' '.join(tokens)

    return text

# Apply preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Show before and after
print("Original text example:", df['text'].iloc[0])
print("\nPreprocessed text example:", df['cleaned_text'].iloc[0])


In [ ]:
# Plot top words for each sentiment
plot_top_words(top_positive, 'Top 10 Words in Positive Sentiment')
plot_top_words(top_negative, 'Top 10 Words in Negative Sentiment')
plot_top_words(top_neutral, 'Top 10 Words in Neutral Sentiment')

In [ ]:
# Create a table of top words
top_words_table = pd.DataFrame({
    'Positive': [word[0] for word in top_positive],
    'Negative': [word[0] for word in top_negative],
    'Neutral': [word[0] for word in top_neutral]
})

top_words_table

In [ ]:
# Check for NaN in labels
print("Number of NaN in labels:", df['label_num'].isna().sum())

# Option 1: Drop rows with NaN labels (recommended)
df = df.dropna(subset=['label_num'])

# Option 2: Fill NaN with a default value (e.g., neutral=1)
# df['label_num'] = df['label_num'].fillna(1)  # Only use if appropriate for your data

# Verify no NaN remains
print("NaN after cleaning:", df['label_num'].isna().sum())

df_deduplicated = df.drop_duplicates(subset=['text', 'label'])
print(f"Removed {len(df) - len(df_deduplicated)} duplicates.")

In [ ]:
# Convert labels to numerical values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label_num'] = df['label'].map(label_map)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'], df['label_num'],
    test_size=0.2, random_state=42, stratify=df['label_num']
)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Dictionary to store model performance
model_results = {}

# List of models to evaluate
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(kernel='linear', probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Naive Bayes': MultinomialNB()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test_tfidf)

    # Store results
    model_results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_test, model.predict_proba(X_test_tfidf), multi_class='ovr')
    }

    # Print classification report
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_map.keys()))

# Convert results to DataFrame for comparison
results_df = pd.DataFrame.from_dict(model_results, orient='index')
print("\nModel Performance Comparison:")
display(results_df.sort_values(by='f1', ascending=False))

# Plot model comparison
plt.figure(figsize=(12, 6))
results_df.sort_values(by='f1', ascending=False).plot(kind='bar', y=['f1', 'accuracy', 'precision', 'recall'])
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()